#Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report,f1_score, confusion_matrix, accuracy_score
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LogisticRegression_base.csv')
df.head()

#Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [ ]:
df['lag_1'] = df['sales_sum'].shift(1)
df['lag_2'] = df['sales_sum'].shift(2)
df['lag_3'] = df['sales_sum'].shift(3)
df['lag_4'] = df['sales_sum'].shift(4)
df['lag_5'] = df['sales_sum'].shift(5)
df['lag_6'] = df['sales_sum'].shift(6)
df['lag_7'] = df['sales_sum'].shift(7)
df['lag_8'] = df['sales_sum'].shift(8)
df['lag_9'] = df['sales_sum'].shift(9)
df['lag_10'] = df['sales_sum'].shift(10)
df['lag_11'] = df['sales_sum'].shift(11)

In [ ]:
df.dropna(inplace=True)
df.head()

In [ ]:
df['is_Sun'] = (df.index.dayofweek == 6).astype(int)
df['is_Mon'] = (df.index.dayofweek == 0).astype(int)
df['is_Tue'] = (df.index.dayofweek == 1).astype(int)
df['is_Wed'] = (df.index.dayofweek == 2).astype(int)
df['is_Thu'] = (df.index.dayofweek == 3).astype(int)
df['is_Fri'] = (df.index.dayofweek == 4).astype(int)
df.head(3)

In [ ]:
df['is_Jan'] = (df.index.month == 1).astype(int)
df['is_Feb'] = (df.index.month == 2).astype(int)
df['is_Mar'] = (df.index.month == 3).astype(int)
df['is_Apr'] = (df.index.month == 4).astype(int)
df['is_May'] = (df.index.month == 5).astype(int)
df['is_Jun'] = (df.index.month == 6).astype(int)
df['is_Jul'] = (df.index.month == 7).astype(int)
df['is_Aug'] = (df.index.month == 8).astype(int)
df['is_Sep'] = (df.index.month == 9).astype(int)
df['is_Oct'] = (df.index.month == 10).astype(int)
df['is_Nov'] = (df.index.month == 11).astype(int)
df['is_Dec'] = (df.index.month == 12).astype(int)
df.head(3)

In [ ]:
df = df.tail(187)
df

In [ ]:
threshold = np.percentile(df['sales_sum'], 66)
threshold

In [ ]:
df['is_hotseller'] = np.where(df['sales_sum'] >= threshold, 1, 0)
df.head()

In [ ]:
# นับจำนวน is_hot_seller ที่เป็น 1 และ 0
is_hotseller_counts = df['is_hotseller'].value_counts()

print("จำนวน is_hotseller ที่เป็น 1:", is_hotseller_counts.get(1, 0))
print("จำนวน is_hotseller ที่เป็น 0:", is_hotseller_counts.get(0, 0))

In [ ]:
df.tail(8)

In [ ]:
# สร้าง Target สำหรับ Direct Method
forecast_horizon = 7
target_cols = []
for h in range(1, forecast_horizon + 1):
    col_name = f'is_hotseller_t+{h}'
    df[col_name] = df['is_hotseller'].shift(-h)
    target_cols.append(col_name)

# กำหนดลิสต์ Feature สุดท้าย
features = [
    'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11'
    , 'time_index', 'year'
    , 'ismarketday', 'isschoolday', 'holiday'
]

In [ ]:
print(df[target_cols])

In [ ]:
# เราจะสร้าง DataFrame ใหม่สำหรับการเทรนโดยการ dropna()
# เพื่อให้ทุกแถวมีข้อมูล X และ y ครบถ้วน
df_for_training = df.dropna()

X_train = df_for_training[features]
y_train = df_for_training[target_cols]

In [ ]:
# 3.1) เตรียม Input (X_test)
# เราจะใช้ Feature จาก "แถวสุดท้ายที่มีข้อมูล X ครบถ้วน" ก่อนที่จะถูก dropna ไป
# ซึ่งก็คือแถวที่ 8 จากท้ายสุดของ DataFrame ดั้งเดิม (index -8)
# แถวนี้คือข้อมูลล่าสุดที่เราสามารถใช้ทำนาย 7 วันข้างหน้าได้
X_test = df.iloc[[-1 - forecast_horizon]][features]

# 3.2) เตรียมค่าจริงเพื่อเปรียบเทียบ (y_test_actual)
# ค่าจริงที่เราจะเปรียบเทียบด้วยคือยอดขาย 7 วันสุดท้ายจริงๆ ของ DataFrame ดั้งเดิม
y_test_actual = df['is_hotseller'].tail(7).values

print(f"\nTest instance to predict from (date: {X_test.index[0].date()})")
print(f"Actual values to compare against (shape: {y_test_actual.shape})")

#Train

มีความจำเป็นต้องกำหนด class_weight='balanced' เนื่องจากปัญหา Imbalance target ซึ่งเกิดจากการต้องการแยกวันสำคัญ ซึ่งคือวันขายดีกว่าปกติ ซึ่งเป็นส่วนน้อย

In [ ]:
base_model = LogisticRegression(class_weight='balanced')
model_direct = MultiOutputClassifier(base_model)

# เทรนโมเดลด้วยข้อมูลที่สมบูรณ์เท่านั้น
model_direct.fit(X_train, y_train)
print("\nDirect multi-output model trained successfully.")

# ทำนาย 7 วันข้างหน้าจาก X_test ที่เราเตรียมไว้
predictions_direct = model_direct.predict(X_test).flatten()
print("Direct 7-step forecast completed.")
print("\nPredicted values:", predictions_direct)

#Evaluate

In [ ]:
# prompt: วัดผลด้วย Confusion Matrix, Classification Report

# แปลงค่า y_test_actual เป็น array ของ integers
y_test_actual = y_test_actual.astype(int)

# แปลง predictions_direct เป็น array ของ integers (ถ้ายังไม่ใช่)
predictions_direct = predictions_direct.astype(int)


# วัดผลด้วย Confusion Matrix
conf_matrix = confusion_matrix(y_test_actual, predictions_direct)
print("\nConfusion Matrix:")
print(conf_matrix)

# แสดง Confusion Matrix แบบสวยงามด้วย heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


# วัดผลด้วย Classification Report
class_report = classification_report(y_test_actual, predictions_direct)
print("\nClassification Report:")
print(class_report)

# คำนวณและแสดง F1-score
f1 = f1_score(y_test_actual, predictions_direct)
print(f"\nF1 Score: {f1:.4f}")

# คำนวณและแสดง Accuracy
accuracy = accuracy_score(y_test_actual, predictions_direct)
print(f"Accuracy Score: {accuracy:.4f}")